# Análisis de Sentimiento

## Corpus de Tweets



In [ ]:
from sentiment.tass import InterTASSReader

reader = InterTASSReader('TASS/InterTASS/tw_faces4tassTrain1000rc.xml')
tweets = list(reader.tweets())  # iterador sobre los tweets
X = list(reader.X())  # iterador sobre los contenidos de los tweets
y = list(reader.y())  # iterador sobre las polaridades de los tweets

In [ ]:
#print(tweets[0].keys())
#print(X[0])
#print(y[0])

## Entrenamiento y Evaluación

In [1]:
%run sentiment/scripts/train.py -m clf -c maxent -o clf_maxent
# %run sentiment/scripts/curve.py -m clf -c maxent

In [2]:
%run sentiment/scripts/eval.py -i clf_maxent

Sentiment P:
  Precision: 55.30% (120/217)
  Recall: 76.92% (120/156)
  F1: 64.34%
Sentiment N:
  Precision: 67.18% (131/195)
  Recall: 59.82% (131/219)
  F1: 63.29%
Sentiment NEU:
  Precision: 15.00% (3/20)
  Recall: 4.35% (3/69)
  F1: 6.74%
Sentiment NONE:
  Precision: 29.73% (22/74)
  Recall: 35.48% (22/62)
  F1: 32.35%
Accuracy: 54.55% (276/506)
Macro-Precision: 41.80%
Macro-Recall: 44.14%
Macro-F1: 42.94%
	P	N	NEU	NONE
P	120	20	7	9	
N	45	131	9	34	
NEU	33	24	3	9	
NONE	19	20	1	22	


## Jugando con el Clasificador

In [4]:
pipeline = model._pipeline
x = X[0]
print(x)
y = pipeline.predict(['estamos todos muy tristes'])
print(y)
#P = pipeline.predict_proba([x])
# D = pipeline.decision_function([x])
# print(D)
# print(x)
# print(y)
# print(P)
# print(type(P))  P es un array de numpy (o sea, una matriz)
#print(pipeline.classes_)

@noseashetero 1000/10 de verdad a ti que voy a decir petarda que te quiero más que a mí mismo  ✨
['N']


## Análisis del Modelo y Errores

Para el clasificador maxent podemos consultar las características que más favorecen o desfavorecen cada clase:

In [5]:
# el model ya quedó cargado al haber corrido eval.py
pipeline = model._pipeline
# norm = pipeline.named_steps['norm']
vect = pipeline.named_steps['vect']
clf = pipeline.named_steps['clf']

from sentiment.analysis import print_maxent_features
print_maxent_features(vect, clf)

N:
	port bes ) enhorabuen maravill ([-1.73672947 -1.68388457 -1.66122784 -1.49959025 -1.49320248])
	dolor mal peor denunci trist ([1.86992474 1.89365963 2.0767024  2.10640275 2.36932745])
NEU:
	salud not_porqu president enhorabuen 6 ([-1.42524089 -1.22799446 -1.09372826 -1.03402328 -0.99906009])
	expect camps perdon convers tranquil ([1.25371167 1.35403923 1.39586498 1.59507114 1.72814477])
NONE:
	encant mal feliz graci apoy ([-2.17367693 -2.09441975 -1.97057239 -1.85226463 -1.83706697])
	not_otr posesion 7714 reunion port ([1.2894653  1.30457447 1.36366543 1.39630086 2.1885628 ])
P:
	blanc urdangarin trist cuent critic ([-1.59395065 -1.53968117 -1.43379717 -1.42313938 -1.39628165])
	graci homenaj maravill encant enhorabuen ([1.96062818 2.13499616 2.20104348 2.47710913 2.62374376])


El script de evaluación también calcula una matriz de confusión detallada "cm_items" para ver en qué instancias falla. Veamos las instancias que son negativas y fueron marcadas como positivas:

In [ ]:
# print(cm_items['N', 'P'])
X2 = [X[i] for i in cm_items['N', 'P']]  # obtenemos los contenidos
print(X2[10])

Usando predict_proba podemos calcular los "peores" errores, esto es, los que más favorecieron P por encima de N:

In [ ]:
P = pipeline.predict_proba(X2)  # calculamos las probabilidades para todas las clases
# print(P.shape)
# print(P[0])
# print(pipeline.classes_)
delta = P[:,3] - P[:,0]  # diferencia entre prob de P y prob de N
# print(delta[0])
# print(delta.shape)
sorted_X2 = sorted(zip(X2, delta), key=lambda x: x[1], reverse=True)   # ordenamos de mayor a menor
sorted_X2[0]

In [ ]:
print(pipeline.predict_proba(['@sport Teneis el ojino como la bandera de Japón ,hijos de la gran puta ']))
# print(pipeline.decision_function(['JA JA JA JA JA']))
pipeline.classes_

¡Acá parece que las repeticiones afectan mucho!

También se puede ver para una instancia particular, qué features tiene y cuánto pesan para cada clase:

In [ ]:
from sentiment.analysis import print_feature_weights_for_item
#x = sorted_X2[0][0]
# print(x)
print_feature_weights_for_item(vect, clf, x)

## Normalización

Veamos por ejemplo cómo eliminar URLs y menciones de usuarios usando expresiones regulares. Tomemos un tweet de ejemplo que tenga ambas cosas:

In [ ]:
x = [x for x in X if 'http' in x and '@' in x][1]
x

In [ ]:
mentions = r'(?:@[^\s]+)'  # una arroba seguida de uno o más caracteres que no son de espaciado
urls = r'(?:https?\://t.co/[\w]+)'  # una URL http o https. \w acepta letras, números y '_'.

import re
re.sub(mentions, '', x)
# re.sub(urls, '', x)

## Negaciones

Veamos una forma simple de manejar las negaciones:

In [ ]:
# x = [x for x in X if 'no' in x.split()][0]
# x
x = 'las tengo pero aún no las he leído . Caerán prontito'


In [ ]:
tokens = x.split()
new_tokens = []
negate = False
for token in tokens:
    if token in ['no', 'tampoco']:
        negate = True
    elif token == '.':
        negate = False
    elif negate:
        token = 'NOT_' + token
    new_tokens.append(token)

' '.join(new_tokens)

## Emojis

Los emojis no deben ser filtrados ya que expresan sentimiento. Veamos 

In [ ]:
# pip install emoji
import emoji
emojis = set(emoji.UNICODE_EMOJI)
x = [x for x in X if emojis & set(x.split())][1]  # buscamos algún ejemplo con emojis
print(x)

In [ ]:
import re
token_pattern = r"(?u)\b\w\w+\b"  # este es el patrón de tokenización que usa el count vectorizer
re.findall(token_pattern, x)

Podemos ver que este tokenizador elimina los emojis y la puntuación. Veamos el tokenizador de NLTK:

In [ ]:
#from nltk import word_tokenize
from nltk.tokenize import TweetTokenizer
tkn = TweetTokenizer()
tkn.tokenize(x)

¡Mejor!